# 08. Human-in-the-Loop (人机交互)

## 课程目标
- 理解人机交互（Human-in-the-Loop）的概念
- 掌握中断（Interrupt）机制
- 实现人工审批流程
- 学习输入验证和确认
- 构建交互式对话系统
- 处理用户输入和反馈

## 核心概念

Human-in-the-Loop是现代AI系统的重要特性：
1. **中断执行**：在关键节点暂停等待人工干预
2. **审批流程**：需要人工确认的决策点
3. **输入验证**：人工确认或修改AI决策
4. **交互对话**：与用户进行多轮对话
5. **反馈循环**：基于人工反馈改进系统

In [ ]:
# 环境准备
from typing import TypedDict, Annotated, List, Dict, Any, Optional
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.memory import MemorySaver
import time
import json
from datetime import datetime
import uuid

print("环境准备完成")

## 1. 基础中断机制

实现基本的中断和人工干预：

In [ ]:
# 人机交互状态
class HumanInteractionState(TypedDict):
    task_description: str
    proposed_action: str
    human_input: Optional[str]
    approval_status: str  # pending, approved, rejected
    execution_log: List[str]
    requires_human_input: bool

# 处理节点
def analyze_task(state: HumanInteractionState) -> HumanInteractionState:
    """分析任务"""
    task = state.get("task_description", "未知任务")
    
    # 模拟AI分析并提出建议
    proposed_action = f"建议对'{task}'执行自动化处理"
    
    print(f"🤖 AI分析: {proposed_action}")
    
    return {
        "proposed_action": proposed_action,
        "approval_status": "pending",
        "execution_log": state.get("execution_log", []) + ["AI分析完成"],
        "requires_human_input": True
    }

def request_human_approval(state: HumanInteractionState) -> HumanInteractionState:
    """请求人工审批"""
    proposed_action = state.get("proposed_action", "")
    
    print(f"👤 请求人工审批: {proposed_action}")
    print("请输入您的决定: [approve/reject/modify]")
    
    # 在实际应用中，这里会暂停执行等待用户输入
    # 这里我们模拟用户输入
    import random
    human_responses = ["approve", "reject", "modify"]
    simulated_input = random.choice(human_responses)
    
    print(f"模拟用户输入: {simulated_input}")
    
    if simulated_input == "approve":
        approval_status = "approved"
        log_message = "人工审批: 通过"
    elif simulated_input == "reject":
        approval_status = "rejected"
        log_message = "人工审批: 拒绝"
    else:
        approval_status = "modify_requested"
        log_message = "人工审批: 要求修改"
    
    return {
        "human_input": simulated_input,
        "approval_status": approval_status,
        "execution_log": state.get("execution_log", []) + [log_message],
        "requires_human_input": False
    }

def execute_approved_action(state: HumanInteractionState) -> HumanInteractionState:
    """执行已批准的动作"""
    proposed_action = state.get("proposed_action", "")
    
    print(f"✅ 执行批准的动作: {proposed_action}")
    
    # 模拟执行
    time.sleep(1)
    
    return {
        "execution_log": state.get("execution_log", []) + ["动作执行完成"]
    }

def handle_rejection(state: HumanInteractionState) -> HumanInteractionState:
    """处理拒绝"""
    print("❌ 动作被拒绝，流程终止")
    
    return {
        "execution_log": state.get("execution_log", []) + ["流程因拒绝而终止"]
    }

def request_modification(state: HumanInteractionState) -> HumanInteractionState:
    """请求修改"""
    print("🔄 请求修改建议")
    print("请提供修改后的建议:")
    
    # 模拟用户提供修改建议
    modified_action = "修改后的动作：执行保守的处理策略"
    print(f"用户修改建议: {modified_action}")
    
    return {
        "proposed_action": modified_action,
        "approval_status": "approved",  # 假设修改后自动批准
        "human_input": modified_action,
        "execution_log": state.get("execution_log", []) + ["用户提供修改建议"]
    }

# 决策函数
def decide_next_step(state: HumanInteractionState) -> str:
    """决定下一步"""
    approval_status = state.get("approval_status", "pending")
    requires_input = state.get("requires_human_input", False)
    
    if requires_input and approval_status == "pending":
        return "request_approval"
    elif approval_status == "approved":
        return "execute"
    elif approval_status == "rejected":
        return "handle_rejection"
    elif approval_status == "modify_requested":
        return "request_modification"
    else:
        return "end"

# 创建人机交互图
def create_human_interaction_graph():
    checkpointer = MemorySaver()
    graph = StateGraph(HumanInteractionState)
    
    # 添加节点
    graph.add_node("analyze", analyze_task)
    graph.add_node("request_approval", request_human_approval)
    graph.add_node("execute", execute_approved_action)
    graph.add_node("handle_rejection", handle_rejection)
    graph.add_node("request_modification", request_modification)
    
    # 设置流程
    graph.set_entry_point("analyze")
    
    # 分析后的条件分支
    graph.add_conditional_edges(
        "analyze",
        decide_next_step,
        {
            "request_approval": "request_approval",
            "execute": "execute",
            "end": END
        }
    )
    
    # 审批后的分支
    graph.add_conditional_edges(
        "request_approval",
        decide_next_step,
        {
            "execute": "execute",
            "handle_rejection": "handle_rejection",
            "request_modification": "request_modification"
        }
    )
    
    # 修改后重新执行
    graph.add_conditional_edges(
        "request_modification",
        decide_next_step,
        {
            "execute": "execute",
            "end": END
        }
    )
    
    # 结束边
    graph.add_edge("execute", END)
    graph.add_edge("handle_rejection", END)
    
    return graph.compile(checkpointer=checkpointer)

# 测试人机交互
interaction_app = create_human_interaction_graph()

print("=== 人机交互演示 ===")

# 测试多个任务
test_tasks = [
    "删除旧数据文件",
    "发送营销邮件",
    "更新系统配置"
]

for i, task in enumerate(test_tasks):
    print(f"\n--- 任务 {i+1}: {task} ---")
    
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}
    
    result = interaction_app.invoke({
        "task_description": task,
        "execution_log": []
    }, config=config)
    
    print(f"\n任务结果:")
    print(f"最终状态: {result.get('approval_status', 'N/A')}")
    print(f"人工输入: {result.get('human_input', 'N/A')}")
    print(f"执行日志: {' -> '.join(result.get('execution_log', []))}")

## 2. 多轮对话系统

构建支持多轮对话的人机交互系统：

In [ ]:
# 对话状态
class ConversationState(TypedDict):
    conversation_id: str
    messages: List[Dict[str, str]]
    current_topic: str
    user_intent: str
    context: Dict[str, Any]
    awaiting_input: bool
    conversation_complete: bool

def start_conversation(state: ConversationState) -> ConversationState:
    """开始对话"""
    conversation_id = f"conv_{int(time.time())}"
    
    welcome_message = "您好！我是AI助手。请告诉我您需要什么帮助？"
    
    print(f"🤖 {welcome_message}")
    
    return {
        "conversation_id": conversation_id,
        "messages": [{"role": "assistant", "content": welcome_message}],
        "awaiting_input": True,
        "conversation_complete": False,
        "context": {}
    }

def process_user_input(state: ConversationState) -> ConversationState:
    """处理用户输入"""
    # 模拟用户输入
    simulated_inputs = [
        "我想了解如何设置邮件自动回复",
        "具体步骤是什么？",
        "谢谢，我明白了"
    ]
    
    # 获取当前对话轮次
    current_turn = len([msg for msg in state.get("messages", []) if msg["role"] == "user"])
    
    if current_turn < len(simulated_inputs):
        user_input = simulated_inputs[current_turn]
    else:
        user_input = "再见"
    
    print(f"👤 用户: {user_input}")
    
    # 分析用户意图
    if "设置" in user_input or "如何" in user_input:
        intent = "help_request"
        topic = "邮件设置"
    elif "步骤" in user_input or "具体" in user_input:
        intent = "detail_request"
        topic = state.get("current_topic", "邮件设置")
    elif "谢谢" in user_input or "明白" in user_input:
        intent = "acknowledgment"
        topic = state.get("current_topic", "邮件设置")
    elif "再见" in user_input:
        intent = "goodbye"
        topic = "结束对话"
    else:
        intent = "general"
        topic = "一般询问"
    
    messages = state.get("messages", [])
    messages.append({"role": "user", "content": user_input})
    
    return {
        "messages": messages,
        "user_intent": intent,
        "current_topic": topic,
        "awaiting_input": False
    }

def generate_response(state: ConversationState) -> ConversationState:
    """生成AI响应"""
    intent = state.get("user_intent", "general")
    topic = state.get("current_topic", "")
    context = state.get("context", {})
    
    # 根据意图生成响应
    if intent == "help_request":
        response = f"我来帮您了解{topic}。邮件自动回复可以在邮箱设置中配置。您需要详细的步骤说明吗？"
        awaiting_input = True
        context["help_provided"] = True
        
    elif intent == "detail_request":
        response = """详细步骤如下：
1. 打开您的邮箱客户端
2. 进入"设置"或"偏好设置"
3. 找到"自动回复"或"外出回复"选项
4. 启用该功能并编写您的自动回复内容
5. 设置生效时间段
6. 保存设置

还有其他问题吗？"""
        awaiting_input = True
        context["details_provided"] = True
        
    elif intent == "acknowledgment":
        response = "很高兴能帮到您！如果后续还有其他问题，随时可以找我。还有什么需要帮助的吗？"
        awaiting_input = True
        
    elif intent == "goodbye":
        response = "再见！祝您使用愉快！"
        awaiting_input = False
        conversation_complete = True
        
    else:
        response = "我理解您的问题。能否请您再详细说明一下，这样我可以更好地帮助您？"
        awaiting_input = True
    
    print(f"🤖 AI: {response}")
    
    messages = state.get("messages", [])
    messages.append({"role": "assistant", "content": response})
    
    return {
        "messages": messages,
        "awaiting_input": awaiting_input,
        "conversation_complete": conversation_complete if intent == "goodbye" else False,
        "context": context
    }

def should_continue_conversation(state: ConversationState) -> str:
    """决定是否继续对话"""
    if state.get("conversation_complete", False):
        return "end"
    elif state.get("awaiting_input", False):
        return "get_input"
    else:
        return "respond"

# 创建对话系统
def create_conversation_graph():
    checkpointer = MemorySaver()
    graph = StateGraph(ConversationState)
    
    graph.add_node("start", start_conversation)
    graph.add_node("get_input", process_user_input)
    graph.add_node("respond", generate_response)
    
    graph.set_entry_point("start")
    
    # 开始后等待输入
    graph.add_conditional_edges(
        "start",
        should_continue_conversation,
        {
            "get_input": "get_input",
            "end": END
        }
    )
    
    # 获取输入后生成响应
    graph.add_conditional_edges(
        "get_input",
        should_continue_conversation,
        {
            "respond": "respond",
            "end": END
        }
    )
    
    # 响应后决定下一步
    graph.add_conditional_edges(
        "respond",
        should_continue_conversation,
        {
            "get_input": "get_input",
            "end": END
        }
    )
    
    return graph.compile(checkpointer=checkpointer)

# 测试对话系统
conversation_app = create_conversation_graph()

print("\n=== 多轮对话系统演示 ===")

conv_thread_id = str(uuid.uuid4())
conv_config = {"configurable": {"thread_id": conv_thread_id}}

# 运行完整对话
conversation_result = conversation_app.invoke({}, config=conv_config)

print(f"\n=== 对话总结 ===")
print(f"对话ID: {conversation_result.get('conversation_id', 'N/A')}")
print(f"消息总数: {len(conversation_result.get('messages', []))}")
print(f"最终话题: {conversation_result.get('current_topic', 'N/A')}")
print(f"对话完成: {conversation_result.get('conversation_complete', False)}")

# 显示完整对话历史
print(f"\n=== 对话历史 ===")
for i, msg in enumerate(conversation_result.get('messages', []), 1):
    role_icon = "🤖" if msg['role'] == 'assistant' else "👤"
    print(f"{i}. {role_icon} {msg['role'].title()}: {msg['content'][:100]}..." if len(msg['content']) > 100 else f"{i}. {role_icon} {msg['role'].title()}: {msg['content']}")

## 3. 实践案例：智能客服工单系统

构建一个完整的智能客服工单处理系统：

In [ ]:
# 工单系统状态
class TicketState(TypedDict):
    ticket_id: str
    customer_issue: str
    issue_category: str
    severity_level: str
    ai_solution: str
    requires_human_review: bool
    human_agent_assigned: Optional[str]
    resolution_status: str
    customer_feedback: Optional[str]
    processing_log: List[str]

def analyze_customer_issue(state: TicketState) -> TicketState:
    """分析客户问题"""
    issue = state.get("customer_issue", "")
    ticket_id = f"TK-{int(time.time())}"[-8:]
    
    # AI分析问题类别和严重程度
    if "无法登录" in issue or "密码" in issue:
        category = "账户问题"
        severity = "中等"
    elif "付款" in issue or "账单" in issue:
        category = "财务问题"
        severity = "高"
    elif "功能" in issue or "使用" in issue:
        category = "功能咨询"
        severity = "低"
    elif "故障" in issue or "错误" in issue:
        category = "技术故障"
        severity = "高"
    else:
        category = "一般咨询"
        severity = "低"
    
    print(f"📋 工单 {ticket_id}: 分析客户问题")
    print(f"   类别: {category}, 严重程度: {severity}")
    
    return {
        "ticket_id": ticket_id,
        "issue_category": category,
        "severity_level": severity,
        "resolution_status": "analyzing",
        "processing_log": ["问题分析完成"]
    }

def generate_ai_solution(state: TicketState) -> TicketState:
    """生成AI解决方案"""
    category = state.get("issue_category", "")
    severity = state.get("severity_level", "")
    
    # 根据类别生成解决方案
    solutions = {
        "账户问题": "请尝试以下步骤：1) 点击'忘记密码'重置密码 2) 清除浏览器缓存 3) 使用其他设备尝试登录",
        "财务问题": "建议：1) 检查账户余额和付款方式 2) 联系银行确认交易状态 3) 如需进一步帮助，将转接财务专员",
        "功能咨询": "使用指南：1) 查看帮助文档 2) 观看教学视频 3) 如需个性化指导，可安排专人协助",
        "技术故障": "故障处理：1) 重启应用程序 2) 检查网络连接 3) 更新到最新版本 4) 如问题持续，技术团队将介入",
        "一般咨询": "感谢您的咨询！我们的客服团队会尽快为您提供详细解答。"
    }
    
    ai_solution = solutions.get(category, "我们正在分析您的问题，请稍候...")
    
    # 判断是否需要人工审核
    requires_human = severity == "高" or category in ["财务问题", "技术故障"]
    
    print(f"🤖 AI方案: {ai_solution[:50]}...")
    print(f"🔍 需要人工审核: {'是' if requires_human else '否'}")
    
    return {
        "ai_solution": ai_solution,
        "requires_human_review": requires_human,
        "resolution_status": "solution_generated",
        "processing_log": state.get("processing_log", []) + ["AI方案生成完成"]
    }

def human_agent_review(state: TicketState) -> TicketState:
    """人工客服审核"""
    ticket_id = state.get("ticket_id", "")
    ai_solution = state.get("ai_solution", "")
    category = state.get("issue_category", "")
    
    # 模拟分配人工客服
    agents = ["Alice", "Bob", "Carol", "David"]
    assigned_agent = random.choice(agents)
    
    print(f"👨‍💼 人工客服 {assigned_agent} 审核工单 {ticket_id}")
    print(f"   审核AI方案: {ai_solution[:80]}...")
    
    # 模拟人工审核决定
    review_decisions = ["approve", "modify", "escalate"]
    decision = random.choice(review_decisions)
    
    if decision == "approve":
        print(f"✅ {assigned_agent}: AI方案已批准")
        status = "approved_by_human"
        log_msg = f"人工客服{assigned_agent}批准AI方案"
        
    elif decision == "modify":
        modified_solution = f"{ai_solution} [人工修改：建议优先联系技术支持团队]"
        print(f"✏️ {assigned_agent}: AI方案已修改")
        status = "modified_by_human"
        log_msg = f"人工客服{assigned_agent}修改了AI方案"
        
    else:  # escalate
        print(f"⬆️ {assigned_agent}: 问题升级至专业团队")
        status = "escalated"
        log_msg = f"人工客服{assigned_agent}将问题升级"
    
    return {
        "human_agent_assigned": assigned_agent,
        "resolution_status": status,
        "ai_solution": modified_solution if decision == "modify" else state.get("ai_solution", ""),
        "processing_log": state.get("processing_log", []) + [log_msg]
    }

def send_solution_to_customer(state: TicketState) -> TicketState:
    """向客户发送解决方案"""
    ticket_id = state.get("ticket_id", "")
    solution = state.get("ai_solution", "")
    
    print(f"📧 向客户发送工单 {ticket_id} 的解决方案")
    print(f"   方案内容: {solution[:100]}...")
    
    # 模拟客户反馈
    feedback_options = ["满意", "部分解决", "未解决"]
    customer_feedback = random.choice(feedback_options)
    
    print(f"📝 客户反馈: {customer_feedback}")
    
    if customer_feedback == "满意":
        status = "resolved"
    elif customer_feedback == "部分解决":
        status = "partially_resolved"
    else:
        status = "unresolved"
    
    return {
        "customer_feedback": customer_feedback,
        "resolution_status": status,
        "processing_log": state.get("processing_log", []) + [f"解决方案已发送，客户反馈：{customer_feedback}"]
    }

def escalate_to_specialist(state: TicketState) -> TicketState:
    """升级给专家处理"""
    ticket_id = state.get("ticket_id", "")
    category = state.get("issue_category", "")
    
    specialists = {
        "技术故障": "技术支持团队",
        "财务问题": "财务专员",
        "账户问题": "账户安全专家",
    }
    
    specialist = specialists.get(category, "高级客服专员")
    
    print(f"🎯 工单 {ticket_id} 已升级至 {specialist}")
    print(f"   预计24小时内获得专业解答")
    
    return {
        "resolution_status": "escalated_to_specialist",
        "human_agent_assigned": specialist,
        "processing_log": state.get("processing_log", []) + [f"升级至{specialist}"]
    }

# 决策函数
def decide_ticket_flow(state: TicketState) -> str:
    """决定工单处理流程"""
    status = state.get("resolution_status", "")
    requires_human = state.get("requires_human_review", False)
    
    if status == "analyzing":
        return "generate_solution"
    elif status == "solution_generated" and requires_human:
        return "human_review"
    elif status == "solution_generated" and not requires_human:
        return "send_solution"
    elif status in ["approved_by_human", "modified_by_human"]:
        return "send_solution"
    elif status == "escalated":
        return "escalate_specialist"
    else:
        return "end"

# 创建工单处理系统
def create_ticket_system():
    checkpointer = MemorySaver()
    graph = StateGraph(TicketState)
    
    graph.add_node("analyze", analyze_customer_issue)
    graph.add_node("generate_solution", generate_ai_solution)
    graph.add_node("human_review", human_agent_review)
    graph.add_node("send_solution", send_solution_to_customer)
    graph.add_node("escalate_specialist", escalate_to_specialist)
    
    graph.set_entry_point("analyze")
    
    # 添加条件边
    for node in ["analyze", "generate_solution", "human_review"]:
        graph.add_conditional_edges(
            node,
            decide_ticket_flow,
            {
                "generate_solution": "generate_solution",
                "human_review": "human_review",
                "send_solution": "send_solution",
                "escalate_specialist": "escalate_specialist",
                "end": END
            }
        )
    
    graph.add_edge("send_solution", END)
    graph.add_edge("escalate_specialist", END)
    
    return graph.compile(checkpointer=checkpointer)

# 测试工单系统
ticket_app = create_ticket_system()

print("\n=== 智能客服工单系统演示 ===")

# 测试不同类型的客户问题
test_issues = [
    "我无法登录账户，一直提示密码错误",
    "付款时遇到错误，订单状态异常",
    "如何使用新功能进行数据导出？",
    "系统出现严重故障，无法正常工作",
    "想了解你们的服务包含哪些内容"
]

for i, issue in enumerate(test_issues):
    print(f"\n{'='*20} 工单 {i+1} {'='*20}")
    print(f"客户问题: {issue}")
    
    ticket_thread_id = str(uuid.uuid4())
    ticket_config = {"configurable": {"thread_id": ticket_thread_id}}
    
    result = ticket_app.invoke({
        "customer_issue": issue
    }, config=ticket_config)
    
    print(f"\n📊 处理结果:")
    print(f"   工单ID: {result.get('ticket_id', 'N/A')}")
    print(f"   问题分类: {result.get('issue_category', 'N/A')}")
    print(f"   严重程度: {result.get('severity_level', 'N/A')}")
    print(f"   最终状态: {result.get('resolution_status', 'N/A')}")
    print(f"   分配客服: {result.get('human_agent_assigned', '无')}")
    print(f"   客户反馈: {result.get('customer_feedback', '未收到')}")
    print(f"   处理步骤: {' -> '.join(result.get('processing_log', []))}")

## 4. 练习题

### 练习1：审批工作流系统
创建一个多级审批工作流系统：

In [ ]:
# 练习1：请实现审批工作流系统
# TODO: 实现多级审批（部门主管、财务、总经理）
# TODO: 支持并行审批和串行审批
# TODO: 处理审批拒绝和修改请求
# TODO: 添加审批时限和自动升级机制

print("请实现审批工作流系统")

### 练习2：智能问答机器人
构建一个支持多轮对话的智能问答机器人：

In [ ]:
# 练习2：请实现智能问答机器人
# TODO: 实现上下文理解和记忆
# TODO: 支持多种问题类型（事实问答、推理、创意等）
# TODO: 添加不确定时的人工转接机制
# TODO: 实现用户满意度评估和反馈收集

print("请实现智能问答机器人")

## 总结

在本课中，我们学习了Human-in-the-Loop的核心概念和实现：

### 关键要点：
1. **中断机制**：在关键决策点暂停等待人工干预
2. **审批流程**：实现多级审批和决策验证
3. **多轮对话**：维护对话上下文和状态
4. **智能路由**：根据复杂度和类型决定处理路径
5. **反馈循环**：收集人工反馈改进系统

### 最佳实践：
- **合理设置中断点**：在真正需要人工判断的地方设置中断
- **清晰的交互界面**：提供直观的人工干预接口
- **超时处理**：设置合理的等待时间和默认行为
- **状态管理**：正确维护交互过程中的状态
- **日志记录**：完整记录人机交互过程

### 应用场景：
- 智能客服系统
- 审批工作流
- 内容审核平台
- 医疗诊断辅助
- 金融风控系统
- 法律咨询服务

## 下一课预告

在下一课《流式输出和实时交互》中，我们将学习：
- 流式输出的实现原理
- 事件流处理机制
- 实时状态更新
- Token级别的流式响应
- 异步执行模式
- WebSocket集成和实时通信